In [1]:
import cv2
import mediapipe as mp
from fer import FER
import face_recognition
import numpy as np
import pyttsx3

# Initialize Mediapipe Hand and Face Mesh
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize FER for facial expression recognition
expression_detector = FER()

# Initialize text-to-speech engine
tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 150)  # Set speech rate
tts_engine.setProperty('volume', 0.9)  # Set volume level

# Define a function to recognize sign language gestures
def recognize_gesture(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    
    thumb_ip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP]
    index_pip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]
    middle_pip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP]
    ring_pip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP]
    pinky_pip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP]

    # Gesture definitions
    gestures = {
        'Hello': (all(finger_tip.y < thumb_ip.y for finger_tip in [thumb_tip, index_tip, middle_tip, ring_tip, pinky_tip])),
        'Yes': (thumb_tip.y < thumb_ip.y and all(finger_tip.y > thumb_ip.y for finger_tip in [index_tip, middle_tip, ring_tip, pinky_tip])),
        'No': (thumb_tip.y > thumb_ip.y and all(finger_tip.y < thumb_ip.y for finger_tip in [index_tip, middle_tip, ring_tip, pinky_tip])),
        'Peace': (index_tip.y < thumb_tip.y and middle_tip.y < thumb_tip.y and ring_tip.y > thumb_tip.y and pinky_tip.y > thumb_tip.y),
        'Thank You': (thumb_tip.x < index_tip.x and np.linalg.norm(np.array([thumb_tip.x, thumb_tip.y]) - np.array([index_tip.x, index_tip.y])) < 0.05 and 
                      all(finger_tip.y > thumb_tip.y for finger_tip in [middle_tip, ring_tip, pinky_tip])),
        'Help': (all(finger_tip.y > pip.y for finger_tip, pip in zip(
            [thumb_tip, index_tip, middle_tip, ring_tip, pinky_tip],
            [thumb_ip, index_pip, middle_pip, ring_pip, pinky_pip]
        )))
    }

    for gesture, condition in gestures.items():
        if condition:
            return gesture

    return None

def detect_features():
    cap = cv2.VideoCapture(0)  # Change the index if your webcam is not at index 0

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5) as hands:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Could not read frame.")
                break

            # Flip the image horizontally for a later selfie-view display
            frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Resize frame to improve speed
            small_frame = cv2.resize(rgb_frame, (640, 480))

            # Detect hands
            hand_results = hands.process(small_frame)
            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    gesture = recognize_gesture(hand_landmarks)
                    if gesture:
                        text_pos = (int(hand_landmarks.landmark[0].x * frame.shape[1]), int(hand_landmarks.landmark[0].y * frame.shape[0]) - 10)
                        cv2.putText(frame, gesture, text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
                        if gesture == "Hello":
                            tts_engine.say("Hello")
                        else:
                            tts_engine.say(gesture)
                        tts_engine.runAndWait()

            # Detect facial expressions
            face_locations = face_recognition.face_locations(small_frame)
            for (top, right, bottom, left) in face_locations:
                face_roi = frame[top:bottom, left:right]
                emotions = expression_detector.detect_emotions(face_roi)
                if emotions:
                    dominant_emotion, emotion_score = max(emotions[0]['emotions'].items(), key=lambda item: item[1])
                    emotion_label = f'{dominant_emotion} ({emotion_score:.2f})'
                    cv2.putText(frame, emotion_label, (left, top - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

            cv2.imshow('RealTimeFeatures', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
detect_features()


INFO:comtypes.client._code_cache:Imported existing <module 'comtypes.gen' from 'C:\\Users\\samee\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\comtypes\\gen\\__init__.py'>
INFO:comtypes.client._code_cache:Using writeable comtypes cache directory: 'C:\Users\samee\AppData\Local\Programs\Python\Python312\Lib\site-packages\comtypes\gen'
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '

